In [1]:
# ArcPy syntax: https://pro.arcgis.com/en/pro-app/tool-reference/data-management/an-overview-of-the-data-management-toolbox.htm
# Workflow: https://gis.stackexchange.com/questions/51929/counting-and-rasterizing-polygon-overlaps-in-arcgis-desktop

# Import libraries
import arcpy
from arcpy import env
from arcpy.sa import *
import os

#Set workspace
arcpy.env.workspace = r'D:\UWT\Capstone\python\land_preferences'

# Set variable for shapefile in workspace
shp = arcpy.ListFeatureClasses()

# Create geodatabase and print message
gdb_name = 'preferences.gdb'
gdb_folder_path = arcpy.env.workspace
arcpy.CreateFileGDB_management(gdb_folder_path, gdb_name)

print('Geodatabase created')

# Import shapefile to geodatabase
gdb = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb'

for shapefile in shp:
    outFeatureClass = os.path.join(gdb, shapefile.strip(".shp"))
    arcpy.CopyFeatures_management(shapefile, outFeatureClass)

print('Shapefile imported to geodatabase')

# Split feature class into separate feature classes by use_type attribute
in_feature_class = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\land_preference'
target_workspace = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb'
field = ['use_type']

arcpy.SplitByAttributes_analysis(in_feature_class, target_workspace, field)

arcpy.Delete_management(in_feature_class)

print('Split by use type complete')

# Union
arcpy.env.workspace = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb'

for fc in arcpy.ListFeatureClasses():
    union_out = arcpy.Describe(fc).baseName + "_Union"
    arcpy.Union_analysis(fc, union_out)
    arcpy.Delete_management(fc)

print('Union complete')

# Dissolve with count
arcpy.env.workspace = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb'

for fc in arcpy.ListFeatureClasses():
    in_features = fc
    out_feature_class = arcpy.Describe(fc).baseName + "_Dissolve"
    arcpy.Dissolve_management(in_features, out_feature_class, "Shape_Length;Shape_Area", "Shape_Length COUNT", "SINGLE_PART", "DISSOLVE_LINES")
    arcpy.Delete_management(fc)

print('Dissolve complete')

# Create rasters
arcpy.env.workspace = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb'

for fc in arcpy.ListFeatureClasses():
    in_features = fc
    field = "COUNT_Shape_Length"
    out_raster = arcpy.Describe(fc).baseName + "_Raster"
    arcpy.FeatureToRaster_conversion(in_features, field, out_raster)
    arcpy.Delete_management(fc)

print('Rasters complete')

#Rename rasters https://community.esri.com/thread/93814
arcpy.env.workspace = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb'
arcpy.ListRasters()
for raster in arcpy.ListRasters():
    if raster.endswith("_Union_Dissolve_Raster"):
        basename = raster.split("_Union_Dissolve_Raster")[0]
        newrastername = basename
        arcpy.CopyRaster_management(raster,newrastername)
        arcpy.Delete_management(raster)
        
print('Rename rasters complete')

# Create conflict potential rasters by land use type (yes/no)
arcpy.env.workspace = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb'
no_conflict = []

conservation = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\Conservation'
no_conservation = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\No_Conservation'
mining = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\Mining'
no_mining = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\No_Mining'
recreation = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\Recreation'
no_recreation = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\No_Recreation'
timber_harvest = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\Timber_Harvest'
no_timber_harvest = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\No_Timber_Harvest'
tourism = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\Tourism'
no_tourism = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\No_Tourism'
grazing_livestock = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\Grazing_Livestock'
no_grazing_livestock = r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\No_Grazing_Livestock'

# agreement score = MAX(MIN(support, oppose), 0.1)/MAX(support, oppose) - (Brown, 2014)
# https://pro.arcgis.com/en/pro-app/tool-reference/data-management/append.htm
try:
    conservation_agreement = Con((Raster(conservation) > Raster(no_conservation)) & (Raster(no_conservation) > 0.1), Raster(no_conservation)/Raster(conservation))
    conservation_agreement.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\conservation_agreement')
    cons2 = Con((Raster(conservation) > Raster(no_conservation)) & (Raster(no_conservation) < 0.1), 0.1/Raster(conservation))
    cons2.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\cons2')
    arcpy.Append_management("cons2", "conservation_agreement", "TEST", None, '', '')
    arcpy.Delete_management(cons2)
    cons3 = Con((Raster(no_conservation) > Raster(conservation)) & (Raster(conservation) > 0.1), Raster(conservation)/Raster(no_conservation))
    cons3.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\cons3')
    arcpy.Append_management("cons3", "conservation_agreement", "TEST", None, '', '')
    arcpy.Delete_management(cons3)
    cons4 = Con((Raster(no_conservation) > Raster(conservation)) & (Raster(conservation) < 0.1), 0.1/Raster(no_conservation))
    cons4.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\cons4')
    arcpy.Append_management("cons4", "conservation_agreement", "TEST", None, '', '')
    arcpy.Delete_management(cons4)
    cons5 = Con((Raster(conservation) == Raster(no_conservation)), Raster(conservation)/Raster(no_conservation))
    cons5.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\cons5')
    arcpy.Append_management("cons5", "conservation_agreement", "TEST", None, '', '')
    arcpy.Delete_management(cons5)
except:
    no_conflict.append('Conservation')
    
try:
    conservation_agreement = Con((Raster(mining) > Raster(no_mining)) & (Raster(no_mining) > 0.1), Raster(no_mining)/Raster(mining))
    conservation_agreement.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\mining_agreement')
    min2 = Con((Raster(mining) > Raster(no_mining)) & (Raster(no_mining) < 0.1), 0.1/Raster(mining))
    min2.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\min2')
    arcpy.Append_management("min2", "mining_agreement", "TEST", None, '', '')
    arcpy.Delete_management(min2)
    min3 = Con((Raster(no_mining) > Raster(mining)) & (Raster(mining) > 0.1), Raster(mining)/Raster(no_mining))
    min3.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\min3')
    arcpy.Append_management("min3", "mining_agreement", "TEST", None, '', '')
    arcpy.Delete_management(min3)
    min4 = Con((Raster(no_mining) > Raster(mining)) & (Raster(mining) < 0.1), 0.1/Raster(no_mining))
    min4.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\min4')
    arcpy.Append_management("min4", "mining_agreement", "TEST", None, '', '')
    arcpy.Delete_management(min4)
    min5 = Con((Raster(mining) == Raster(no_mining)), Raster(mining)/Raster(no_mining))
    min5.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\min5')
    arcpy.Append_management("min5", "mining_agreement", "TEST", None, '', '')
    arcpy.Delete_management(min5)
except:
    no_conflict.append('Mining')

try:    
    recreation_agreement = Con((Raster(recreation) > Raster(no_recreation)) & (Raster(no_recreation) > 0.1), Raster(no_recreation)/Raster(recreation))
    recreation_agreement.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\recreation_agreement')
    rec2 = Con((Raster(recreation) > Raster(no_recreation)) & (Raster(no_recreation) < 0.1), 0.1/Raster(recreation))
    rec2.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\rec2')
    arcpy.Append_management("rec2", "recreation_agreement", "TEST", None, '', '')
    arcpy.Delete_management(rec2)
    rec3 = Con((Raster(no_recreation) > Raster(recreation)) & (Raster(recreation) > 0.1), Raster(recreation)/Raster(no_recreation))
    rec3.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\rec3')
    arcpy.Append_management("rec3", "recreation_agreement", "TEST", None, '', '')
    arcpy.Delete_management(rec3)
    rec4 = Con((Raster(no_recreation) > Raster(recreation)) & (Raster(recreation) < 0.1), 0.1/Raster(no_recreation))
    rec4.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\rec4')
    arcpy.Append_management("rec4", "recreation_agreement", "TEST", None, '', '')
    arcpy.Delete_management(rec4)
    rec5 = Con((Raster(recreation) == Raster(no_recreation)), Raster(recreation)/Raster(no_recreation))
    rec5.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\rec5')
    arcpy.Append_management("rec5", "recreation_agreement", "TEST", None, '', '')
    arcpy.Delete_management(rec5)
except:
    no_conflict.append('Recreation')

try:
    timber_harvest_agreement = Con((Raster(timber_harvest) > Raster(no_timber_harvest)) & (Raster(no_timber_harvest) > 0.1), Raster(no_timber_harvest)/Raster(timber_harvest))
    timber_harvest_agreement.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\timber_harvest_agreement')
    tim2 = Con((Raster(timber_harvest) > Raster(no_timber_harvest)) & (Raster(no_timber_harvest) < 0.1), 0.1/Raster(timber_harvest))
    tim2.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tim2')
    arcpy.Append_management("tim2", "timber_harvest_agreement", "TEST", None, '', '')
    arcpy.Delete_management(tim2)
    tim3 = Con((Raster(no_timber_harvest) > Raster(timber_harvest)) & (Raster(timber_harvest) > 0.1), Raster(timber_harvest)/Raster(no_timber_harvest))
    tim3.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tim3')
    arcpy.Append_management("tim3", "timber_harvest_agreement", "TEST", None, '', '')
    arcpy.Delete_management(tim3)
    tim4 = Con((Raster(no_timber_harvest) > Raster(timber_harvest)) & (Raster(timber_harvest) < 0.1), 0.1/Raster(no_timber_harvest))
    tim4.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tim4')
    arcpy.Append_management("tim4", "timber_harvest_agreement", "TEST", None, '', '')
    arcpy.Delete_management(tim4)
    tim5 = Con((Raster(timber_harvest) == Raster(no_timber_harvest)), Raster(timber_harvest)/Raster(no_timber_harvest))
    tim5.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tim5')
    arcpy.Append_management("tim5", "timber_harvest_agreement", "TEST", None, '', '')
    arcpy.Delete_management(tim5)
except:
    no_conflict.append('Timber Harvest')
    
try:
    tourism_agreement = Con((Raster(tourism) > Raster(no_tourism)) & (Raster(no_tourism) > 0.1), Raster(no_tourism)/Raster(tourism))
    tourism_agreement.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tourism_agreement')
    tour2 = Con((Raster(tourism) > Raster(no_tourism)) & (Raster(no_tourism) < 0.1), 0.1/Raster(tourism))
    tour2.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tour2')
    arcpy.Append_management("tour2", "tourism_agreement", "TEST", None, '', '')
    arcpy.Delete_management(tour2)
    tour3 = Con((Raster(no_tourism) > Raster(tourism)) & (Raster(tourism) > 0.1), Raster(tourism)/Raster(no_tourism))
    tour3.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tour3')
    arcpy.Append_management("tour3", "tourism_agreement", "TEST", None, '', '')
    arcpy.Delete_management(tour3)
    tour4 = Con((Raster(no_tourism) > Raster(tourism)) & (Raster(tourism) < 0.1), 0.1/Raster(no_tourism))
    tour4.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tour4')
    arcpy.Append_management("tour4", "tourism_agreement", "TEST", None, '', '')
    arcpy.Delete_management(tour4)
    tour5 = Con((Raster(tourism) == Raster(no_tourism)), Raster(tourism)/Raster(no_tourism))
    tour5.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\tour5')
    arcpy.Append_management("tour5", "tourism_agreement", "TEST", None, '', '')
    arcpy.Delete_management(tour5)
except:
    no_conflict.append('Tourism')

try:
    grazing_agreement = Con((Raster(grazing_livestock) > Raster(no_grazing_livestock)) & (Raster(no_grazing_livestock) > 0.1), Raster(no_grazing_livestock)/Raster(grazing_livestock))
    grazing_agreement.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\grazing_agreement')
    graze2 = Con((Raster(grazing_livestock) > Raster(no_grazing_livestock)) & (Raster(no_grazing_livestock) < 0.1), 0.1/Raster(grazing_livestock))
    graze2.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\graze2')
    arcpy.Append_management("graze2", "grazing_agreement", "TEST", None, '', '')
    arcpy.Delete_management(graze2)
    graze3 = Con((Raster(no_grazing_livestock) > Raster(grazing_livestock)) & (Raster(grazing_livestock) > 0.1), Raster(grazing_livestock)/Raster(no_grazing_livestock))
    graze3.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\graze3')
    arcpy.Append_management("graze3", "grazing_agreement", "TEST", None, '', '')
    arcpy.Delete_management(graze3)
    graze4 = Con((Raster(no_grazing_livestock) > Raster(grazing_livestock)) & (Raster(grazing_livestock) < 0.1), 0.1/Raster(no_grazing_livestock))
    graze4.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\graze4')
    arcpy.Append_management("graze4", "grazing_agreement", "TEST", None, '', '')
    arcpy.Delete_management(graze4)
    graze5 = Con((Raster(grazing_livestock) == Raster(no_grazing_livestock)), Raster(grazing_livestock)/Raster(no_grazing_livestock))
    graze5.save(r'D:\UWT\Capstone\python\land_preferences\preferences.gdb\graze5')
    arcpy.Append_management("graze5", "grazing_agreement", "TEST", None, '', '')
    arcpy.Delete_management(graze5)
except:
    no_conflict.append('Livestock Grazing')

arcpy.Delete_management(conservation)
arcpy.Delete_management(no_conservation)
arcpy.Delete_management(mining)
arcpy.Delete_management(no_mining)
arcpy.Delete_management(recreation)
arcpy.Delete_management(no_recreation)
arcpy.Delete_management(timber_harvest)
arcpy.Delete_management(no_timber_harvest)
arcpy.Delete_management(tourism)
arcpy.Delete_management(no_tourism)
arcpy.Delete_management(grazing_livestock)
arcpy.Delete_management(no_grazing_livestock)

print('Conflict rasters complete')
print('No Conflict: ' + str(no_conflict))

Geodatabase created
Shapefile imported to geodatabase


AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Split by use type complete
Union complete
Dissolve complete
Rasters complete
Rename rasters complete
Conflict rasters complete
No Conflict: ['Tourism']
